In [1]:
###ALL SOLUTIONS####

In [2]:
from collections import defaultdict

def getpuzzle(day):
    return open(f'2018day{day}_input.txt','r').read().split('\n')

In [3]:
#DAY 1
from collections import defaultdict

def day1():
    puzzle = getpuzzle(1)
    changes = [int(x) for x in puzzle]
#     print('part 1:',sum(changes))
    s = sum(changes)
    
    freqs = [0]
    modgroups = [0]
    it = [0]
    for c in changes:
        freqs.append(freqs[-1]+c)
        modgroups.append(freqs[-1]%sum(changes))
    
    #check for repeat in first iteration
    fset = set()
    for f in freqs:
        if f in fset: return s,f
        fset.add(f)
        
    if s == 0: return s,0 #0 sum edge case
    
    freqs = freqs[:-1] #last number belongs in second iteration
    mg = defaultdict(list)
    for i,m in enumerate(freqs):
        mg[m % s].append((i,m)) #key is mod loop value, value is index and value
    
    mindiff = None
    for g in mg.values():

        sorted_g = list(sorted(g, key=lambda t: t[1])) #ensure ascending order
        for i in range(1,len(sorted_g)):
            #diff is index, value, iterations to repeat
            diff = (sorted_g[i][0], sorted_g[i][1], (sorted_g[i][1]-sorted_g[i-1][1])//s)
            if mindiff is None or diff[2]<mindiff[2] or (diff[2]==mindiff[2] and diff[0]<mindiff[0]):
                mindiff = diff

    return s,mindiff[1]


answer = day1()
print(f'part 1: {answer[0]}\npart 2: {answer[1]}')

In [4]:
#DAY 2

puzzle = getpuzzle(2)

csum = [0,0]
for id in puzzle:
    cts = [id.count(l) for l in id]
    csum = [csum[0]+(cts.count(2)>0),csum[1]+(cts.count(3)>0)]
print('part 1:',csum[0]*csum[1])

t = puzzle
for i in range(len(t)):
    for j in range(i+1,len(t)):
        matches = [t[i][l]==t[j][l] for l in range(len(t[i]))]
        if matches.count(False)==1:
            print('part 2:',''.join([t[i][l] for l in range(len(t[i])) if matches[l]==True]))

In [5]:
#DAY 3

puzzle = getpuzzle(3)

inches = defaultdict(list)

for claim in puzzle:
    id = int(claim.split(' ')[0][1:])
    l,t = claim.replace(':','').split(' ')[2].split(',')
    w,h = claim.split(' ')[3].split('x')
    for i in range(int(w)):
        for j in range(int(h)):
            inches[int(l)+i,int(t)+j].append(id)

print('part 1:', sum([len(j)>1 for i,j in inches.items()]))


dset = set()
for _,ids in inches.items():
    if len(ids)>1:
        [dset.add(id) for id in ids]

for claim in puzzle:
    id = int(claim.split(' ')[0][1:])
    if id not in dset: print('part 2:',id)


part 1: 118223
part 2: 412


In [6]:
#DAY 4

puzzle = getpuzzle(4)

events = sorted(puzzle)
prev_day,prev_minute = None,None
full = defaultdict(list)
guards_by_minute = {}

for i in range(len(events)):
    #handle shifts starting before midnight
    if events[i][12:14] == '23':
        minute = '00:00'
        if events[i][6:8] != events[i+1][6:8] and events[i+1][9:11]=='02':
            day = events[i+1][6:9]+'01'
        else:
            day = events[i+1][6:11]
    else:
        day = events[i][6:11]
        minute = events[i][12:17]
    #handle start of shift
    if events[i][-5:]=='shift': 
        guard = events[i].split(' ')[3]
        full[day] = {
            'guard': guard,
            'minutes': [0]*60
        }
        if guard not in guards_by_minute: guards_by_minute[guard] = [0]*60
    
    #add asleep minutes if same shift and prior event is sleep event
    if day == prev_day and events[i][-6:] != 'asleep':
        for i in range(int(prev_minute[-2:]), int(minute[-2:])): full[day]['minutes'][i]=1
        for i in range(int(prev_minute[-2:]), int(minute[-2:])): guards_by_minute[guard][i]+=1
    
    prev_day,prev_minute = day, minute


    
    
p1_id = sorted(guards_by_minute.items(), key=lambda x: sum(x[1]))[-1][0]
p1_minute = guards_by_minute[p1_id].index(max(guards_by_minute[p1_id]))

print(f'part 1: guard {p1_id} sleeps most at {p1_minute}\npart 1 answer: {int(p1_id[1:])*p1_minute}')

p2_id = sorted(guards_by_minute.items(), key=lambda x: max(x[1]))[-1][0]
p2_minute = guards_by_minute[p2_id].index(max(guards_by_minute[p2_id]))
print(f'part 2: guard {p2_id} sleeps most at {p2_minute}\npart 2 answer: {int(p2_id[1:])*p2_minute}')

part 1: guard #2953 sleeps most at 39
part 1 answer: 115167
part 2: guard #1069 sleeps most at 30
part 2 answer: 32070


In [10]:
#DAY 5
puzzle = getpuzzle(5)[0]

i = 0
def reactPolymer(base):
    if type(base)==list:
        t = base.copy()
    else: 
        t = base
    i=0
    while i<len(t)-1:
        if i < 0: i=0
        if abs(ord(t[i])-ord(t[i+1]))==32:
            t=t[:i]+t[i+2:]
            i-=2
        i+=1
    return t

s = reactPolymer(puzzle)
print(f'part 1: reduces {len(puzzle)} to {len(s)}')


units = [x for x in set(s) if chr(ord(x)-32) not in set(s)]
results = {}
for u in units:
    exclude = (ord(u),ord(u)+32)
    unit = str([chr(x) for x in exclude])[2:-2:5]
    t = [x for x in s if ord(x) not in exclude]
    results[unit] = (len(t),len(collapsePolymer(t)))

p2 = sorted(results.items(),key=lambda x:x[1][1])[0]

print(f'part 2: removing {list(p2[0])} reduces {p2[1][0]} further to {p2[1][1]}')

part 1: reduces 50000 to 11264
part 2: removing ['G', 'g'] reduces 10900 further to 4552


In [11]:
#DAY 6
puzzle = getpuzzle(6)
puzzle[:5]

['77, 279', '216, 187', '72, 301', '183, 82', '57, 170']

In [18]:
# sorted(puzzle,key=lambda x: x[0])
sorted([x.split(', ') for x in puzzle],key=lambda x: int(x[0]))

[['40', '52'],
 ['43', '294'],
 ['46', '335'],
 ['50', '129'],
 ['55', '89'],
 ['57', '170'],
 ['61', '52'],
 ['66', '158'],
 ['71', '114'],
 ['72', '301'],
 ['73', '287'],
 ['77', '279'],
 ['78', '136'],
 ['82', '88'],
 ['89', '204'],
 ['92', '112'],
 ['104', '84'],
 ['108', '97'],
 ['117', '129'],
 ['125', '249'],
 ['134', '211'],
 ['141', '264'],
 ['150', '130'],
 ['156', '281'],
 ['181', '270'],
 ['183', '82'],
 ['186', '177'],
 ['187', '332'],
 ['188', '145'],
 ['190', '258'],
 ['193', '342'],
 ['210', '271'],
 ['216', '187'],
 ['234', '321'],
 ['237', '278'],
 ['238', '216'],
 ['243', '160'],
 ['254', '292'],
 ['260', '288'],
 ['260', '184'],
 ['270', '287'],
 ['283', '353'],
 ['286', '302'],
 ['290', '109'],
 ['291', '170'],
 ['299', '184'],
 ['309', '130'],
 ['313', '358'],
 ['339', '314'],
 ['353', '107']]

In [20]:
sorted([int(y) for y in x.split(', ') for x in puzzle],key=lambda x: int(x[1]))

NameError: name 'x' is not defined